<a href="https://colab.research.google.com/github/rsenoi/imersaoalura/blob/main/Agentes_de_Medicamento_Imers%C3%A3o_IA_Alura_%2B_Google_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Agentes de Medicamento**


Quem nunca se viu perdido após uma consulta médica, sem lembrar exatamente para que serve aquele medicamento receitado? A proposta aqui é criar uma ferramenta capaz de fornecer informações cruciais sobre um dado remédio, como sua indicação terapêutica, modo de uso, a faixa de preço praticada no mercado, possíveis efeitos colaterais, a disponibilidade de um correspondente genérico e a respectiva variação de preço.

Aspectos Técnicos:

Entrada:
 *   Nome do medicamento

Saída:
*   Nome do medicamento completo
*   Indicação terapêutica
*   Posologia e modo de uso
*   Faixa de preço do medicamento original
*   Possíveis efeitos colaterais
*   Disponibilidade de genérico: Sim/Não
*   Faixa de preço do medicamento genérico (se disponível)

Arquitetura:

Agente 1, Agente 2 => Redator => Revisor
*   Agente de Busca ANVISA:
Encarregado de consultar a base de dados da Agência Nacional de Vigilância Sanitária (ANVISA) para obter informações oficiais sobre o medicamento, como indicação, uso e efeitos colaterais.

*   Agente de Busca de Preços:
Responsável por pesquisar em diversos sites de e-commerce e farmácias online para coletar dados sobre a faixa de preço do medicamento original dos últimos 5 meses e, se disponível, do genérico.
*   Redator:
Utiliza as informações coletadas pelos Agentes de Busca para gerar um descritivo claro e acessível para o usuário leigo, evitando termos técnicos complexos.
*   Revisor:
Realiza uma verificação ortográfica e de "temperatura" (garantindo que a linguagem seja adequada e amigável) do texto produzido pelo Redator antes de
ser apresentado ao usuário.


In [2]:
%pip -q install google-genai

In [3]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [4]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [5]:
# Instalar Framework ADK de agentes do Google ################################################
%pip install -q google-adk

In [6]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [7]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [8]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [77]:
##########################################
#   Agente 1: Buscador do Medicamento    #
##########################################
def agente_buscador_medicamento(medicamento):
    buscador_medicamento = Agent(
        name="agente_busca_medicamento",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente #################################################
        instruction="""
        Você como um farmaceutico experiente em medicamentos, falando em português, que deve buscar no site oficial da ANVISA, www.gob.br/anvisa,
        no sistema “Bulário Eletronico” os dados do medicamento solicitado
        (Nome do medicamento completo, Indicação, Uso, Efeitos colaterais e se existe medicamento similar generico com o nome)
        Sempre incluir os links de busca oficial da ANVISA utilizado por você como fonte de referência para futuras buscas. Quando não existir o link passe o link da ANVISA com a Bulário Eletronico.
        """,
        description="Agente que busca o descritivo do medicamento",
        tools=[google_search]

    )

    entrada_do_agente_buscador_medicamento = f"Medicamento:{medicamento}\n"
    # Executa o agente
    descritivo_do_medicamento = call_agent(buscador_medicamento, entrada_do_agente_buscador_medicamento)
    return descritivo_do_medicamento

In [79]:
################################################
# --- Agente 2: buscador de preco --- #
################################################
def agente_buscador_preco(medicamento):
    buscador_preco = Agent(
        name="agente_buscador_preco",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Buscador #################################################
        instruction="""
        Você como um comprador de produtos farmacêuticos experiente, falando em português, deve buscar nos sites de e-commerce brasileiros o medicamento
        contendo o menor e o maior preço em reais, dos últimos 5 meses. Quando existir preço, coloque os preços no formato "R$ 0,00 " ou "menor preço R$ 0,00 e maior preço R$ 0,00"
        Inclua os links de busca dos e-commerces pesquisados por você para servirem como referência de busca.
        """,
        description="Agente que busca o preco",
        tools=[google_search]
    )

    entrada_do_agente_buscador_preco = f"Medicamento:{medicamento}\n"
    # Executa o agente
    preco_medicamento = call_agent(buscador_preco, entrada_do_agente_buscador_preco)
    return preco_medicamento

In [62]:
######################################
# --- Agente 3: Redator das buscas - #
######################################
def agente_redator(medicamento, descritivo_do_medicamento, preco_medicamento):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.5-flash-preview-04-17",
        instruction="""
            Você como um farmaceutico experiente em atendimento de pessoas leigas em medicamentos e que responde de forma resumida, para que todos entendam,
            falando em português, resuma as informações do medicamento(descritivo e preco) contendo os campos: Nome do remedio, Indicação, Uso, Faixa de preço, Efeitos colaterais, Generico e Faxia preço do genérico.
            Não inclua seus comentários pessoais, somente as informações do medicamento.
            No campo “Uso”, após a narrativa inclua um adendo para verificar na receita o tempo de utilização. Separa os campos "Genérico" e "Faixa de preço do genérico".
            No campo “Genérico" e no campo "Faixa de preço do generico”, inclua o nome do medicamento similar e preco caso existam.
            Inclua, ao final de cada pesquisa o campo “Observação: ” com a seguinte premissa:
            Sempre deixar claro que as informações descritas não substituem a ida ao médico.
            No campo “Source” sempre colocar os links de busca passados no descritivo e no preços. Caso não exista a link da ANVISA incluir como referencia.
            Ao invés de “Source” coloque “Referências”.
            """,
        description="Agente redator de medicadmentos ira resumir as informacoes"
    )
    entrada_do_agente_redator = f"Medicamento: {medicamento}\nDescricao do Medicamento: {descritivo_do_medicamento}\nPreco: {preco_medicamento}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [58]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(medicamento, rascunho):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.5-flash-preview-04-17",
        instruction="""
            Você é um Editor e Farmaceutico Revisor de Conteúdo meticuloso, especializado em medicamentos, com foco em bulas.
            Por ter um público leigo em medicamentos, que fala em português e de forma resumida, revise o rascunho, verificando clareza, concisão, correção e tom.
            Retorne o texto caso estaja adequado com a "Descritivo: APROVADO" no inicio do texto, pulando uma linha para o inicio do texto.
            Caso haja problemas, corrija e retorne o texto com a "Descritivo: REVISADO" no inicio do texto, pulando uma linha para o inicio do texto.
            """,
        description="Agente revisor de medicamentos."
    )
    entrada_do_agente_revisor = f"Medicamento: {medicamento}\nRascunho: {rascunho}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [44]:
def impressao(texto):
  print(f"\n---------------------------------\n")
  display(to_markdown(texto))
  print(f"\n---------------------------------\n\n")

In [ ]:
# Chamada Principal  ################################################
medicamento = input('Digite o nome do medicamento: ')

if medicamento == '':
  print(f'\nMedicamento não informado. Saindo da consulta.\n\n')
else:
  print(f'\nProcessando ... Aguarde...\n')
  descritivo_do_medicamento = agente_buscador_medicamento(medicamento)
  #impressao(descritivo_do_medicamento)
  preco_medicamento = agente_buscador_preco(medicamento)
  #impressao(preco_medicamento)
  rascunho = agente_redator(medicamento, descritivo_do_medicamento, preco_medicamento)
  #impressao(rascunho)
  texto_revisado = agente_revisor(medicamento, rascunho)
  impressao(texto_revisado)



